In [2]:
# ======================================================
# 📊 PROJETO CANNOLI – Preparação de Dados de Campanhas
# ======================================================

# OBJETIVO:
# Demonstrar o processo de preparação e integração de dados
# entre campanhas e filas de envio (Campaign e CampaignQueue)
# usando Python e Pandas, com visualização exploratória.

# ======================================================
# 1. IMPORTAÇÃO DE BIBLIOTECAS E LEITURA DOS DADOS
# ======================================================

import pandas as pd
import numpy as np
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

# Carregar arquivos CSV
campanhas = pd.read_csv("Campaign_semicolon.csv", sep=";", dtype=str)
envios = pd.read_csv("CampaignQueue_semicolon.csv", sep=";", dtype=str)

print("✅ Arquivos carregados com sucesso!")
print(f"Campanhas: {campanhas.shape}")
print(f"Envios: {envios.shape}")

# ======================================================
# 2. LIMPEZA E ORGANIZAÇÃO DOS DADOS
# ======================================================

# Padronizar nomes de colunas
for df in [campanhas, envios]:
    df.columns = df.columns.str.strip().str.lower()

# Converter datas (aceita formatos diferentes)
def ajustar_datas(df):
    for c in df.columns:
        if "at" in c or "date" in c:
            df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

campanhas = ajustar_datas(campanhas)
envios = ajustar_datas(envios)

# Remover duplicatas
campanhas.drop_duplicates(inplace=True)
envios.drop_duplicates(inplace=True)

# Padronizar texto
for col in ["name", "badge", "storeid"]:
    if col in campanhas.columns:
        campanhas[col] = campanhas[col].astype(str).str.title()

# Conversão de tipos
if "status" in campanhas.columns:
    campanhas["status"] = pd.to_numeric(campanhas["status"], errors="coerce")

if "status" in envios.columns:
    envios["status"] = pd.to_numeric(envios["status"], errors="coerce")

print("🧹 Dados limpos e prontos para uso!")

# ======================================================
# 3. CRIAÇÃO DE VARIÁVEIS DERIVADAS
# ======================================================

# Indicador de resposta
envios["tem_resposta"] = envios["response"].astype(str).apply(
    lambda x: 0 if x.lower() in ["nan", "none", "null", ""] else 1
)

# Extração de hora do envio
if "sendat" in envios.columns:
    envios["hora_envio"] = envios["sendat"].dt.hour

# Classificação de status das campanhas
status_map = {1: "Rascunho", 2: "Cancelada", 3: "Ativa", 4: "Concluída"}
campanhas["status_legivel"] = campanhas["status"].map(status_map).fillna("Desconhecido")

print("⚙️ Variáveis derivadas criadas com sucesso!")

# ======================================================
# 4. INTEGRAÇÃO ENTRE CAMPANHAS E ENVIOS
# ======================================================

# Normalizar IDs
for col in ["id", "campaignid"]:
    if col in envios.columns:
        envios[col] = envios[col].astype(str).str.strip()
    if col in campanhas.columns:
        campanhas[col] = campanhas[col].astype(str).str.strip()

# Juntar tabelas
base_completa = envios.merge(
    campanhas, left_on="campaignid", right_on="id", how="left", suffixes=("", "_camp")
)

print("🔗 Integração concluída! Total de registros:", base_completa.shape[0])

# ======================================================
# 5. FORMATAÇÃO FINAL
# ======================================================

colunas_finais = [
    "campaignid", "name", "badge", "type", "status_legivel",
    "sendat", "tem_resposta", "hora_envio"
]

df_final = base_completa[[c for c in colunas_finais if c in base_completa.columns]].copy()

df_final.rename(columns={
    "name": "Campanha",
    "badge": "Categoria",
    "type": "Tipo",
    "status_legivel": "Status",
    "tem_resposta": "Resposta Recebida",
    "hora_envio": "Hora de Envio"
}, inplace=True)

print("📁 Dataset final preparado com sucesso!")
display(df_final.head())

# ======================================================
# 6. ANÁLISE EXPLORATÓRIA E VISUALIZAÇÃO
# ======================================================


# 6.2 Taxa de Resposta Geral
taxa_resposta = df_final["Resposta Recebida"].mean() * 100
print(f"💬 Taxa geral de resposta: {taxa_resposta:.2f}%")

# 6.3 Top 10 Campanhas com Maior Engajamento
top_campanhas = (
    df_final.groupby("Campanha")["Resposta Recebida"]
    .mean()
    .reset_index()
    .sort_values("Resposta Recebida", ascending=False)
    .head(10)
)
top_campanhas["Resposta (%)"] = (top_campanhas["Resposta Recebida"] * 100).round(1)

fig2 = px.bar(top_campanhas, x="Campanha", y="Resposta (%)",
              title="💡 Top 10 Campanhas com Maior Taxa de Resposta",
              color="Resposta (%)", color_continuous_scale="Teal")
fig2.update_layout(xaxis_tickangle=-30)
fig2.show()

# 6.4 Engajamento por Hora
if "Hora de Envio" in df_final.columns:
    engajamento_hora = (
        df_final.groupby("Hora de Envio")["Resposta Recebida"]
        .mean().reset_index()
    )
    engajamento_hora["Taxa (%)"] = (engajamento_hora["Resposta Recebida"] * 100).round(1)
    fig3 = px.line(engajamento_hora, x="Hora de Envio", y="Taxa (%)",
                   title="⏰ Engajamento por Hora de Envio",
                   markers=True)
    fig3.show()

# ======================================================
# 7. EXPORTAÇÃO DOS DADOS PRONTOS
# ======================================================

df_final.to_csv("dados_campanhas_envios_tratados.csv", index=False)
print("✅ Arquivo 'dados_campanhas_envios_tratados.csv' exportado com sucesso!")


✅ Arquivos carregados com sucesso!
Campanhas: (2000, 14)
Envios: (5000, 16)
🧹 Dados limpos e prontos para uso!
⚙️ Variáveis derivadas criadas com sucesso!
🔗 Integração concluída! Total de registros: 5000
📁 Dataset final preparado com sucesso!


,campaignid,Campanha,Categoria,Tipo,Status,sendat,Resposta Recebida,Hora de Envio
0,1553,Campanha Odio Izbd,Winback,2,Desconhecido,2025-01-23 22:23:00,0,22.0
1,1890,Campanha Ut T53X,Nan,1,Desconhecido,2025-04-25 02:52:00,1,2.0
2,429,Campanha Et Do5N,Consumption,2,Desconhecido,NaT,0,NaN
3,766,Campanha Tempora 8P3L,Migration,2,Desconhecido,2025-07-03 15:30:00,0,15.0
4,1039,Campanha Adipisci Ybz8,Loyalty,1,Desconhecido,NaT,0,NaN


💬 Taxa geral de resposta: 31.76%


✅ Arquivo 'dados_campanhas_envios_tratados.csv' exportado com sucesso!
